In [1]:
class Grammar:
    def __init__(self):
        # Simple Vietnamese grammar rules
        self.rules = {
            'S': [['NP', 'VP']],
            'NP': [['DET', 'N'], ['N'], ['NP', 'PP']],
            'VP': [['V', 'NP'], ['V']],
            'PP': [['P', 'NP']],
            'DET': ['một', 'cái', 'những'],
            'N': ['tôi', 'cô gái', 'chàng trai', 'quả táo', 'bóng đá', 'sách'],
            'V': ['ăn', 'đọc', 'yêu', 'chơi'],
            'P': ['với', 'trong', 'bằng'],
        }

    def is_nonterminal(self, symbol):
        return symbol in self.rules and isinstance(self.rules[symbol], list)

    def productions(self, nonterminal):
        return self.rules.get(nonterminal, [])

In [ ]:
from collections import defaultdict

# === NGỮ PHÁP TIẾNG VIỆT MỞ RỘNG ===
VN_GRAMMAR_IMPROVED = {
    "S": [["NP", "VP"]],

    # Noun phrases
    "NP": [["Pro"], ["N"], ["Det", "N"], ["N", "N"]],

    # Verb phrases
    "VP": [["V"], ["V", "NP"], ["V", "Adj"], ["V", "PP"], ["Adv", "V"], ["V", "V"]],

    # Prepositional phrases
    "PP": [["P", "NP"]],

    # Pronouns
    "Pro": [["Tôi"], ["Em"], ["Anh"], ["Chị"], ["Bạn"], ["Bố"], ["Mẹ"], ["Ông"], ["Bà"], ["Bé"], ["Con"]],

    # Nouns
    "N": [
        ["cơm"], ["bài"], ["chợ"], ["sách"], ["Hà Nội"], ["cà phê"], ["phim"],
        ["mèo"], ["chó"], ["cờ"], ["tranh"], ["búp bê"], ["nước"], ["báo"], ["mì"],
        ["xe"], ["bánh"], ["nhạc"], ["thư"], ["cây"], ["đồ"], ["tivi"], ["truyện"],
        ["sữa"], ["Toán"], ["tiếng Anh"], ["đồng hồ"], ["ảnh"], ["phòng"], ["hoa"],
        ["đàn"], ["bộ"], ["chén"], ["trưa"], ["dạo"], ["điện"], ["ăn"], ["học"], ["làm"]
    ],

    # Determiners
    "Det": [["Con"], ["Ông"]],

    # Verbs
    "V": [
        ["ăn"], ["học"], ["đi"], ["đọc"], ["nấu"], ["yêu"], ["thích"], ["xem"],
        ["ngủ"], ["sủa"], ["chơi"], ["uống"], ["chạy"], ["sửa"], ["làm"], ["nghe"],
        ["mua"], ["viết"], ["tưới"], ["giặt"], ["ngồi"], ["gọi"], ["chờ"], ["rửa"],
        ["dọn"], ["chụp"], ["thăm"], ["lái"], ["làm việc"], ["hát"], ["vẽ"],
        ["chăm"], ["dạo"], ["bộ"]
    ],

    # Adjectives
    "Adj": [["nhanh"], ["trưa"]],

    # Adverbs
    "Adv": [["đi"]],

    # Prepositions
    "P": [["bộ"]]
}

# === TRẠNG THÁI CHART PARSER ===
class State:
    def __init__(self, lhs, rhs, dot, start, end):
        self.lhs = lhs
        self.rhs = rhs
        self.dot = dot
        self.start = start
        self.end = end

    def next_symbol(self):
        return self.rhs[self.dot] if self.dot < len(self.rhs) else None

    def is_complete(self):
        return self.dot >= len(self.rhs)

    def advance(self):
        return State(self.lhs, self.rhs, self.dot + 1, self.start, self.end)

    def __eq__(self, other):
        return (self.lhs, self.rhs, self.dot, self.start, self.end) == \
               (other.lhs, other.rhs, other.dot, other.start, other.end)

    def __hash__(self):
        return hash((self.lhs, tuple(self.rhs), self.dot, self.start, self.end))

    def __repr__(self):
        before_dot = ' '.join(self.rhs[:self.dot])
        after_dot = ' '.join(self.rhs[self.dot:])
        return f"[{self.start}:{self.end}] {self.lhs} → {before_dot} • {after_dot}"

# === HÀM PHÂN TÍCH CÚ PHÁP ===
def parse_vietnamese(sentence, grammar=VN_GRAMMAR_IMPROVED):
    words = sentence.strip().split()
    n = len(words)
    chart = [set() for _ in range(n + 1)]

    for rhs in grammar["S"]:
        chart[0].add(State("S", rhs, 0, 0, 0))

    for i in range(n + 1):
        added = True
        while added:
            added = False
            states = list(chart[i])
            for state in states:
                next_sym = state.next_symbol()

                if next_sym in grammar:
                    for prod in grammar[next_sym]:
                        new_state = State(next_sym, prod, 0, i, i)
                        if new_state not in chart[i]:
                            chart[i].add(new_state)
                            added = True

                elif i < n and next_sym == words[i]:
                    new_state = State(state.lhs, state.rhs, state.dot + 1, state.start, i + 1)
                    chart[i + 1].add(new_state)

                elif state.is_complete():
                    for st in chart[state.start]:
                        if st.next_symbol() == state.lhs:
                            new_state = State(st.lhs, st.rhs, st.dot + 1, st.start, i)
                            if new_state not in chart[i]:
                                chart[i].add(new_state)
                                added = True

    # Kiểm tra kết quả
    success = False
    for state in chart[n]:
        if state.lhs == "S" and state.start == 0 and state.end == n and state.is_complete():
            success = True
            break

    return success, chart

# === 50 CÂU TIẾNG VIỆT ===
sentences = [
    "Tôi ăn cơm", "Em học bài", "Chị đi chợ", "Bố đọc sách", "Mẹ nấu ăn",
    "Tôi yêu Hà Nội", "Anh thích cà phê", "Bạn xem phim", "Con mèo ngủ",
    "Con chó sủa", "Ông bà chơi cờ", "Tôi hát", "Em vẽ tranh", "Bé chơi búp bê",
    "Tôi uống nước", "Anh đọc báo", "Em chạy nhanh", "Chị nấu mì", "Bố sửa xe",
    "Mẹ làm bánh", "Tôi nghe nhạc", "Bạn làm bài", "Anh đi học", "Em mua sách",
    "Chị chơi đàn", "Tôi đi bộ", "Anh rửa chén", "Em viết thư", "Bố tưới cây",
    "Mẹ giặt đồ", "Tôi ngủ trưa", "Em xem tivi", "Chị đi xe", "Anh học Toán",
    "Tôi đi làm", "Bạn học tiếng Anh", "Em đọc truyện", "Anh uống sữa",
    "Chị thăm bà", "Tôi đi dạo", "Bạn ngồi học", "Mẹ gọi tôi", "Em xem đồng hồ",
    "Tôi chờ xe", "Anh lái xe", "Chị làm việc", "Bạn gọi điện", "Tôi chụp ảnh",
    "Em dọn phòng", "Chị chăm hoa"
]

# === CHẠY PARSER CHO 50 CÂU ===
print("=== KIỂM TRA 50 CÂU TIẾNG VIỆT ===\n")
passed = 0
failed_sentences = []

for s in sentences:
    success, _ = parse_vietnamese(s)
    if success:
        print(f"✔ THÀNH CÔNG: \"{s}\"")
        passed += 1
    else:
        print(f"✖ THẤT BẠI: \"{s}\"")
        failed_sentences.append(s)

print(f"\n➡️ Đã phân tích thành công: {passed}/{len(sentences)} câu")

if failed_sentences:
    print("\n=== CÁC CÂU CHƯA PHÂN TÍCH ĐƯỢC ===")
    for s in failed_sentences:
        print(f"- \"{s}\"")

# === PHÂN TÍCH CÂU KHÓ ===
def analyze_sentence(sentence):
    """Phân tích chi tiết câu và hiển thị trạng thái phân tích"""
    print(f"\n=== PHÂN TÍCH CHI TIẾT: \"{sentence}\" ===")
    words = sentence.strip().split()
    success, chart = parse_vietnamese(sentence)

    for i, states in enumerate(chart):
        print(f"\nCHART[{i}]:")
        for state in sorted(states, key=lambda s: (s.start, s.lhs)):
            print(f"  {state}")

    if success:
        print(f"\n✔ THÀNH CÔNG: Câu \"{sentence}\" được phân tích thành công")
    else:
        print(f"\n✖ THẤT BẠI: Câu \"{sentence}\" không thể phân tích")

# Phân tích một số câu khó
problem_sentences = ["Ông bà chơi cờ", "Mẹ nấu ăn", "Tôi đi bộ", "Tôi đi làm"]
for sentence in problem_sentences:
    analyze_sentence(sentence)